In [15]:
import cospaper as cosp
import DynVel as dv
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
reload(dv)
plt.rc('text', usetex=True)
vfdl = pd.read_pickle('../Products/vflis.pickle')
vfdh = pd.read_pickle('../Products/vfhis.pickle')

In [6]:
lisv = vfdl[1]
hisv = vfdh[1]
# Because only optically thick in Si II 1260:
lisv.MeanProfile = lisv.Si_II_1260_Flam
lisv.MeanErrors = lisv.Si_II_1260_Stddev

In [16]:
def randomize(indata, rng=None):
    """Creates a fake dataset by adding a random sample from a gaussian 
    modelled over the actual error distribution to the original data.
    Use result for monte carlo error determination.
    rng: array-like
         a list,  tuple or array of (min, max) of the velocity interval 
         in which to sample the errors.
    """
    # For convenience
    data = indata.MeanProfile
    errs = indata.MeanErrors
    if rng is None:
        rng = [indata.Velocity.min(), indata.Velocity.max()]
    errs = errs[indata.Velocity.between(rng[0], rng[1])]
    # Find distribution of the errors.
    errmean = errs.mean()
    errdev = errs.std()
    # Draw random sample from error distribution
    newerrs = sp.random.normal(errmean, errdev, data.shape[0])
    # Draw random sequence of +/-
    signs = sp.around(sp.random.random(data.shape[0]))
    signs[signs==0] = -1
    # Add or subtract,  randomly chosen, the errrors from/to the measurements
    newerrs *= signs
    indata.MeanErrors = newerrs
    indata.MeanProfile += newerrs
    #indata = indata[indata.Velocity.between(rng[0], rng[1])]
    #print rng
    return indata



def get_ranges(indata):
    spedata = indata.MeanProfile
    wl = indata.Velocity
    fig = plt.figure()  # figsize=(3.4, 3.4), dpi=160)
    ax = plt.subplot(111)
    exes = []
    plt.plot(wl, spedata, 'black', drawstyle='steps-mid')
    plt.axis((-800, 800, -.1, 1.4))
    plt.axhline(y=1., linestyle='--', color='k')
    plt.annotate(r'''Mark piece of left continuum, beginning and end of line, \\
                 and piece of right continuum, 6 clicks in total''',
                 (.5, .9), xycoords='axes fraction',
                 ha='center')
    #cid = fig.canvas.mpl_connect('button_press_event', onclick)
    #plt.show()
    exes = plt.ginput(6)
    plt.close()
    #import pdb; pdb.set_trace() ### XXX BREAKPOINT
    #plt.show()
    # print exes
    leftexes = np.asarray(exes[0:2])[:, 0]
    riteexes = np.asarray(exes[4:6])[:, 0]
    # print 'riteexes', riteexes
    lineexes = np.asarray(exes[2:4])[:, 0]
    exesarr = np.array([leftexes, lineexes, riteexes])
    return exesarr
    
def DVMC(indata, ranges=None, iterations=500):
    import sys
    if ranges is not None:
        rng = [ranges[1, 0], ranges[1, 1]]
    else:
        ranges = get_ranges(indata)
        rng = [ranges[1, 0], ranges[1, 1]]
    #data = indata.MeanProfile
    #errs = indata.MeanErrors
    # Real values:
    dvdict = dv.dynvel(indata, ranges=ranges, plot=True)
    # Errors:
    errsdict = {i: [] for i in dvdict.keys()}
    count = 0
    for j in range(iterations):
        sys.stdout.write("\rIteration:  {}, II0min = {}".format(j, dvdict['I/I0_min']))
        sys.stdout.flush()
        mcdata = indata.copy()
        mcdata = randomize(mcdata, rng=rng)
        # Because some configurations with large errors break down also make the code break down.
        # This is kinda cheating, but shouldn't be too much. Only discards configurations oof too 
        try: 
            mcdict = dv.dynvel(mcdata, ranges=ranges, plot=False)
        except:
            print('Iteration {} failed'.format(j))
            continue
        count += 1
        #print('{}'.format(mcdict))
        for j in mcdict.keys():
            errsdict[j].append(mcdict[j])
    errframe = pd.DataFrame.from_dict(errsdict)
    for quant in errframe.columns:
        err = errframe[quant].std()
        dvdict[quant+'_Std'] = err
    dvdict['count'] = count
    return dvdict, errframe#, errsdict

In [17]:
reload(cosp)
reload(dv)
velsdf = {}
#ranges = get_ranges(vfd[6])

ranges = get_ranges(lisv)
itr = 1000
dvdict, errframe = DVMC(lisv, ranges=ranges, iterations=itr)
#velsdf[galaxy] = dvdict

Iteration:  999, II0min = 0.352843549235

In [18]:
lisvels = pd.DataFrame.from_dict(dvdict, orient='index')
lisvels.columns = ['H11C LIS']
lisvels

,H11C LIS
I/I0_0,0.537561
delta_v_Std,15.404860
count,1000.000000
delta_v,480.343273
v_min_Std,33.166626
v_int,-148.596561
I/I0_min,0.352844
v_95_Std,12.492482
I/I0_min_Std,0.018607
I/I0_0_Std,0.037000


In [9]:
velsdf = {}
#ranges = get_ranges(vfd[6])

ranges = get_ranges(hisv)
itr = 1000
dvdict, errframe = DVMC(hisv, ranges=ranges, iterations=itr)
#velsdf[galaxy] = dvdict
hisvels = pd.DataFrame.from_dict(dvdict, orient='index')
hisvels.columns = ['H11C HIS']
hisvels

Iteration:  999, II0min = 0.150584243952

,H11C HIS
I/I0_0,0.312360
delta_v_Std,71.508067
count,1000.000000
delta_v,820.180187
v_min_Std,44.009739
v_int,-149.989310
I/I0_min,0.150584
v_95_Std,70.642351
I/I0_min_Std,0.023159
I/I0_0_Std,0.104911


In [12]:
(1. - lisv.Si_II_CovFrac_map).plot()
plt.show()

In [ ]:
dv.